In [ ]:
import numpy as np
import vpython as vp

from random import choice
from vpython import *

In [ ]:
# Define a function to map a 1D array to a 2D/3D lattice, only works on lattice of equal-lateral size in Cartesian system
def map(index, size, dimension): 
    if dimension == 1:
        return [index, 0, 0];
    if dimension == 2:
        x = index%size;
        y = np.floor(index/size);
       # return vp.vector(x,y,0);
        return [x, y, 0];
    if dimension == 3:
        sub_index = index%(size**2);
        x = sub_index%size;
        y = np.floor(sub_index/size);
        z = np.floor(index/size**2);
       # return vp.vector(x,y,z)
        return [x, y, z];

In [ ]:
# Define a function to map the 3D vector back into the 1D array index
def antimap(array, size, dimension):
    if dimension == 1:
        return array[0];
    if dimension == 2:
        return array[0] + array[1]*size;
    if dimension == 3:
        return array[0] + array[1]*size + array[2]*size*size;

In [ ]:
# Define a function to find the index of the nearest neighbours
def nNbr(index, size, dimension):
    if dimension == 1:
        return [(index+size-1)%size, (index+size+1)%size]; # apply periodic condition
    
    if dimension == 2:
        pos = map(index,size,dimension);
        north = np.mod(np.add(pos,[0,size+1,0]), size);
        south = np.mod(np.add(pos,[0,size-1,0]), size);
        east = np.mod(np.add(pos,[size+1,0,0]), size);
        west = np.mod(np.add(pos,[size-1,0,0]),size);
        return [antimap(north, size, 2), antimap(south, size, 2), antimap(east, size, 2), antimap(west, size, 2)];
    
    if dimension == 3:
        pos = map(index,size,dimension);
        north = np.mod(np.add(pos,[0,size+1,0]), size);
        south = np.mod(np.add(pos,[0,size-1,0]), size);
        east = np.mod(np.add(pos,[size+1,0,0]), size);
        west = np.mod(np.add(pos,[size-1,0,0]),size);
        up = np.mod(np.add(pos,[0,0,size+1]), size);
        down = np.mod(np.add(pos,[0,0,size-1]), size);
        return [antimap(north, size, 3), antimap(south, size, 3), antimap(east, size, 3), antimap(west, size, 3), antimap(up, size, 3), antimap(down, size, 3)];

In [ ]:
# Set up the system parameters
temp = 100; # in [K] 
L = 30; # Define the lattice size
dims = 2; # Define the dimensionality of the lattice
N = L**dims; # Compute the total number of the sites
kB = 8.617E-5 # Boltzmann constant in [eV/K]
J = -10.0*kB; # Define the coupling strength between spins
beta = 1/(temp*kB);
MaxIteration = 1000; # Define the maximum amount of iteration
r = 0.2; # dot radius for visualization

In [ ]:
# Set up array containers for data manipulation and initial conditions
#lattice = np.empty((N*N,3),int); # Create a container for the lattice sites
Apos = [None]*N; # Create a container for site positions using vpython.vector()
#spin = np.onesN; # Create a container for the spins for all the lattice sites
spinV = np.empty(N,int); # Create a container for the spins for all the lattice sites
spinS = [None]*N; # Create a container to store vpython objects
nIdx = np.empty((N,dims*2), np.intp); # Create a container to store the indices of the nearest neighbours of each site
probs = np.empty(N,np.float); # Create a container to store in each iteration spin-flip probability
for ii in range(N):
    #lattice[ii] = [ii%N, np.floor((ii)/N), 0];
    Apos[ii] = vp.vector(map(ii,L,dims)[0]-L/2, map(ii,L,dims)[1]-L/2, map(ii,L,dims)[2]-L/2);
    spinV[ii] = 1; # Create a container to store the values of the spins
    nIdx[ii] = nNbr(ii, L, dims);
    spinS[ii] = vp.sphere(pos=Apos[ii], radius=r, color=vp.vector(1,0,0)); # Visualize all the spins

In [ ]:
# # Randomization of the initial condition
for ii in range(int(N/2)): # Randomly choose half of the lattice to flip the spin 
    kk = choice(range(N));
    spinV[kk] = -1;
    spinS[kk].color = vp.vector(1,1,1); # color all the spin down as white

In [ ]:
# Start the iteration and implement Metropolis algorithm
tempSpin = spinV; # Create a temporary container to store updated spins
it = 0; # Create an iterator
scene.autoscale = True
while it <= MaxIteration:
    rate(1)
    ruler = np.random.rand(len(spinV));
    for ii in range(len(spinV)):
        nIdx[ii] = nNbr(ii, L, dims);
        probs[ii] = exp(beta*J*sum(spinV[nIdx[ii]])*spinV[ii])/exp(-beta*J*sum(spinV[nIdx[ii]])*spinV[ii]);
        tempSpin[ii] = -spinV[ii] if probs[ii] > ruler[ii] else spinV[ii];
    spinV = tempSpin;
    for ii in range(len(spinV)):
        spinS[ii].color = vp.vector(1,0,0) if spinV[ii] > 0 else vp.vector(1,1,1);
    it = it + 1;